In [1]:
import requests
from requests_oauthlib import OAuth1
import os, json
from time import sleep


url = 'https://api.twitter.com/1.1/account/verify_credentials.json'
auth = OAuth1('E5eeHHPrKm0N8qkHhH1BOkSu6', 'iLkXetVRI93Xt32KJfY44CUfau1BgH4BHSotaDLHloaykLOmTR',
                  '744995423308374017-ic5GQxFdUxYjgpFJSbAQ1rRaJluz4nJ', 'hZl4AAiu62CalzpY8BeJfX2wJXezROk0WsEhms9KxXQ4w')
requests.get(url, auth=auth)


searchWord = input("Please enter a search term: ")
filepath = searchWord.replace(" ","").lower() 
searchWord = searchWord.replace(" ", "%20")
print("This is the search term: " + searchWord)
print("Path:" + filepath)


Please enter a search term: acmilan
This is the search term: acmilan
Path:acmilan


In [2]:
import os
from datetime import datetime
getTweet = 'https://api.twitter.com/1.1/search/tweets.json?q=%40'+searchWord+'&count=50'
#print(requests.get(getTweet, auth=auth))
#print("URL: " + getTweet)
#response = requests.get(getTweet, auth=auth)


#print(data)
#json_str = json.dumps(data)
#data = json.loads(json_str)
# Writing JSON data

#if not os.path.exists(path):
    #os.makedirs(path)

def merge(query):
    data = {}
    while len(data) < 50:
        response = requests.get(getTweet, auth = auth) 
        response = response.json()
        for status in response['statuses']:
            data[status['id']] = status 
        sleep(1)
        print(len(data))
    return data

data = merge(getTweet)
#datetime.now().strftime('%H-%M')+ datetime.datetime.now().strftime("%B") + dattime.now().strftime('%d%Y')+'.json'
with open(filepath+'.json', 'w') as f:
     json.dump(data, f)
        


50


In [3]:
print(len(data))

50


In [6]:
#TimeZones
with open(filepath+'.json', 'r') as fr:
    data = json.load(fr)
time_zone_all = {}
for id in data:
    user = data[id]['user']
    if user['time_zone'] is not None: 
        if user['time_zone'] in time_zone_all:
            time_zone_all[user['time_zone']] += 1
        else:
            time_zone_all[user['time_zone']] = 1

print(time_zone_all)
print(sorted(time_zone_all, key=lambda x: time_zone_all[x]))

top10 = sorted(time_zone_all, key=lambda x: time_zone_all[x])
print(top10[:10])

{'Amsterdam': 1, 'Mountain Time (US & Canada)': 1, 'Bogota': 2, 'Mexico City': 3, 'London': 4, 'Kuala Lumpur': 1, 'Bangkok': 2, 'Pacific Time (US & Canada)': 9, 'Arizona': 1, 'Eastern Time (US & Canada)': 2, 'Rome': 1, 'Jakarta': 8, 'Madrid': 1, 'Central Time (US & Canada)': 1}
['Amsterdam', 'Mountain Time (US & Canada)', 'Kuala Lumpur', 'Arizona', 'Rome', 'Madrid', 'Central Time (US & Canada)', 'Bogota', 'Bangkok', 'Eastern Time (US & Canada)', 'Mexico City', 'London', 'Jakarta', 'Pacific Time (US & Canada)']
['Amsterdam', 'Mountain Time (US & Canada)', 'Kuala Lumpur', 'Arizona', 'Rome', 'Madrid', 'Central Time (US & Canada)', 'Bogota', 'Bangkok', 'Eastern Time (US & Canada)']


In [7]:
#emoticons
emoticons = []
for id in data:
    text = data[id]['text']
    emoticons.append(text.split())    

#print(emoticons)

list_of_codes = ['U+1F601','U+1F602','U+1F603','U+1F604','U+1F605','U+1F606','U+1F607','U+1F6018','U+1F609','U+1F60A'] 
matching = []
for emo in emoticons:
    #for em in emo:
     #   if "hey" in em:
      #      matching.append[em]
    #matching += [token for token in emo if token.startswith('\U0001')]
    matching += [token for token in emo if 'hey' in token]
    
print(matching)
print(len(matching))

[]
0


In [8]:
print("\U0001F600")

😀


In [9]:
#pre processing
import nltk
import re
#nltk.download('punkt')

from nltk.tokenize import word_tokenize

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
 

tweets_tokenized = []
for id in data:
    tweet = data[id]['text']
    tweets_tokenized.append(preprocess(tweet))


print(tweets_tokenized)

[['RT', '@acmilan', ':', 'We', 'are', 'sure', 'you', 'are', 'still', 'smiling', 'after', '#MilanJuve', ',', 'wherever', 'you', 'are', '.', 'Ciao', 'Sic', ',', "we'll", 'never', 'stop', 'missing', 'you', '❤', '️', '⚫', '️', 'https://t.co', '…'], ['RT', '@affai_', ':', '@acmilan', 'Spirit', 'of', 'Monday', '.', '#MilanJuve', 'https://t.co/aE6qz0WSfC'], ['RT', '@davidecalabria2', ':', 'Caro', 'vecchio', 'cuore', 'rossonero', 'che', 'emozioni', 'che', 'ci', 'dai', '❤', '️', '🔴', '⚫', '️', '@acmilan', 'https://t.co/NYjqWkOpbs'], ['Lo', 'que', 'voy', 'a', 'estrenar', 'ese', 'martes', '@acmilan', 'para', 'el', '#GenoaMilan', '😍', 'https://t.co/2tyb2WI26S'], ['RT', '@AliHarb83', ':', '@ACMClubMTL', '@BarCiociaro', '@acmilan', 'nice', 'to', 'meet', 'u', 'too', 'you', 'guys', 'are', 'awesome', '!', 'Haha', 'what', 'an', 'honor', '.', "I'll", 'make', 'sure', 'to', 'keep', 'those', '…'], ['@DemetrioAlbe', '@MilanEye', '@Vivo_Azzurro', '@acmilan', '@juventusfc', '@MilanTV', '@MilanNewsit'], ['RT', 

In [10]:
import operator 
import json
from collections import Counter
 
count_all = Counter()
for line in tweets_tokenized:
    if line not in stop:
        count_all.update(line)
print(count_all.most_common(5))


[('@acmilan', 48), ('!', 41), ('.', 39), (':', 38), ('RT', 34)]


In [41]:
#to remove stop words
import nltk
from nltk.corpus import stopwords
#nltk.download()
from collections import Counter
 

import string
 
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']
count_all = Counter()

for line in tweets_tokenized:
    for term in line:
        if term not in stop:
            terms_stop.append(term)
    #terms_stop = [term for term in line if term not in stop]
    #terms_stop.append(terms_stop)
            count_all.update(line)

# for word in terms_stop:
#     count_all += 1
# print(count_all)
print(count_all.most_common(5))

 
print(terms_stop[:100])

[('.', 783), ('@acmilan', 782), ('!', 658), (':', 655), ('RT', 580)]
['RT', '@Info4_Milanisti', 'FT', '@acmilan', '1', 'vs', '0', '@juventusfc', '64', 'Manuel', 'Locatelli', '#MilanJuve', '#SerieA', '#ForzaMilan', '🔴', '⚫', ['RT', '@Info4_Milanisti', 'FT', '@acmilan', '1', 'vs', '0', '@juventusfc', '64', 'Manuel', 'Locatelli', '#MilanJuve', '#SerieA', '#ForzaMilan', '🔴', '⚫', [...], 'RT', '@acmilan', 'We', 'sure', 'still', 'smiling', '#MilanJuve', 'wherever', 'Ciao', 'Sic', "we'll", 'never', 'stop', 'missing', '❤', '️', '⚫', '️', 'https://t.co', '…', 'RT', '@affai_', '@acmilan', 'Spirit', 'Monday', '#MilanJuve', 'https://t.co/aE6qz0WSfC', 'RT', '@davidecalabria2', 'Caro', 'vecchio', 'cuore', 'rossonero', 'che', 'emozioni', 'che', 'ci', 'dai', '❤', '️', '🔴', '⚫', '️', '@acmilan', 'https://t.co/NYjqWkOpbs', 'Lo', 'que', 'voy', 'estrenar', 'ese', 'martes', '@acmilan', 'para', 'el', '#GenoaMilan', '😍', 'https://t.co/2tyb2WI26S', 'RT', '@AliHarb83', '@ACMClubMTL', '@BarCiociaro', '@acmilan'

In [18]:
terms_stop = [term for term in preprocess(tweet['text']) if term not in stop]

TypeError: string indices must be integers

In [46]:
cnt = Counter()
for word in ['RT', '@Info4_Milanisti', 'FT', '@acmilan', '1', 'vs', '0', '@juventusfc', '64', 'Manuel', 'Locatelli', '#MilanJuve', '#SerieA', '#ForzaMilan', '🔴', '⚫',
            ['RT', '@Info4_Milanisti', 'FT', '@acmilan', '1', 'vs', '0', '@juventusfc', '64', 'Manuel', 'Locatelli', '#MilanJuve', '#SerieA', '#ForzaMilan', '🔴', '⚫', [...],
            'RT', '@acmilan', 'We', 'sure', 'still', 'smiling', '#MilanJuve', 'wherever', 'Ciao', 'Sic', "we'll", 'never', 'stop', 'missing', '❤', '️', '⚫', '️', 'https://t.co', '…', 'RT',
             '@affai_', '@acmilan', 'Spirit', 'Monday', '#MilanJuve', 'https://t.co/aE6qz0WSfC', 'RT', '@davidecalabria2', 
             'Caro', 'vecchio', 'cuore', 'rossonero', 'che', 'emozioni', 'che', 'ci', 'dai', '❤', '️', '🔴', '⚫', '️',
             '@acmilan', 'https://t.co/NYjqWkOpbs', 'Lo', 'que', 'voy', 'estrenar', 
             'ese', 'martes', '@acmilan', 'para', 'el', '#GenoaMilan', '😍', 'https://t.co/2tyb2WI26S', 'RT']]:
             
    cnt[word] += 1
cnt

TypeError: unhashable type: 'list'

In [45]:
import datetime
mydate = datetime.datetime.now()
mydate.strftime("%B")

print(mydate.strftime("%B"))

October


In [93]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
 
    
for everytweet in tweets_tokenized:
    print(preprocess(everytweet))
#tweet = 'RT @marcobonzanini: just an example! :D http://example.com #NLP'
print(tweets_tokenized)
# ['RT', '@marcobonzanini', ':', 'just', 'an', 'example', '!', ':D', 'http://example.com', '#NLP']

TypeError: expected string or bytes-like object

In [88]:
import nltk
#nltk.download('punkt')
from nltk.tokenize import word_tokenize

sentence = """At eight o'clock on Thursday morning
... Arthur didn't feel very good."""
tokens = nltk.word_tokenize(sentence)
print(tokens)


['At', 'eight', "o'clock", 'on', 'Thursday', 'morning', '...', 'Arthur', 'did', "n't", 'feel', 'very', 'good', '.']


In [89]:
import nltk
#nltk.download('punkt')
from nltk.tokenize import word_tokenize

sentence = """At eight o'clock on Thursday morning
... Arthur didn't feel very good."""
tokens = nltk.word_tokenize(sentence)
print(tokens)


['At', 'eight', "o'clock", 'on', 'Thursday', 'morning', '...', 'Arthur', 'did', "n't", 'feel', 'very', 'good', '.']
